In [1]:
from collections import defaultdict
import numpy as np
import sys
from enum import Enum
class ACTION(Enum):
    high='high'
    low='low' 
    
class Winner:
    def __init__(self):
        self.energy_max = 10
        self.energy_add = 2
        self.energy_prob = 0.2
        self.P_H = 0.55
        self.P_L = 0.45
        self.state = 0
        self.done = 0
        self.R = 0
        self.highcost = 1
        self.lowcost = 0
        self.greedy = 0.1
        self.energy=self.energy_max
        self.discount=0.9
        self.init_Q()
        return
    def reset_game(self):
        self.state=0
        self.done=0
        self.energy=self.energy_max
        return
    def ba_policy(self,state,energy):
        smp = np.random.uniform(0,1)
        if smp > self.greedy:
            if self.Q[state,energy,ACTION.high]>self.Q[state,energy,ACTION.low]:
                ac=ACTION.high
            else:
                ac=ACTION.low
        else:
            sp = np.random.uniform(0,1)
            if sp > 0.5:
                ac=ACTION.high
            else:
                ac=ACTION.low
        return ac
        
    #def ta_policy(self):
    def sarsalam(self,numepisode,lam=0.9,stepsize=0.001):
        self.init_Q()
        for i_episode in range(1, numepisode+1):
            if i_episode % 1000 == 0:
                print("\rEpisode {}/{}.".format(i_episode, numepisode), end="")
                sys.stdout.flush()
            
            self.init_Etrace()
            self.reset_game()
            
            action=self.ba_policy(state=self.state,energy=self.energy)
            
            while(1):
                reward,nextstate,nextenergy,self.done=self.env_step(self.state,self.energy,action)
                
                
                nextaction = self.ba_policy(state=nextstate,energy=nextenergy)
                
                
                delta = reward + self.discount*self.Q[nextstate,nextenergy,nextaction] - self.Q[self.state,self.energy,action]
                #print('delta',delta)
                self.Etrace[self.state,self.energy,action] += 1
                
                #print(self.Etrace[self.state,self.energy,action])
                for state_t in range (7):
                    for act_t in ACTION:
                        for energy_t in range(1,self.energy_max+1):
                            self.Q[state_t-3,energy_t,act_t]+=stepsize*delta*self.Etrace[state_t-3,energy_t,act_t]
                            
                            self.Etrace[state_t-3,energy_t,act_t] = self.discount*lam*self.Etrace[state_t-3,energy_t,act_t]
                self.state = nextstate
                action = nextaction
                self.energy=nextenergy
                if self.done==1:
                    break
                
        return self.Q
        
        
    def env_step(self,state,energy,action):
        done=0
        reward=0
        sample = np.random.uniform(0, 1)
        smp = np.random.uniform(0,1)

        if action == ACTION.high:
            if energy >= self.highcost :
                suc_prob = self.P_H
                energy = energy - self.highcost
            else:
                suc_prob = self.P_L
                reward = reward + self.lowcost
        else: 
            suc_prob = self.P_L
            reward = reward + self.lowcost
        if sample < suc_prob:
            state = state + 1
        else:
            state = state -1
        if state >= 3 :
            reward = reward + 1000
            done = 1
        if state <= (-3):
            done = 1
        if smp < self.energy_prob:
            energy = energy + self.energy_add
        if energy > self.energy_max:
            energy = self.energy_max
        #print(state)
        return reward,state,energy,done
    def init_Q(self):
        self.Q = defaultdict(lambda:np.zeros(1))
        for state in range(7):
            for act in ACTION:
                for energy in range(1,self.energy_max+1):
                    self.Q[state-3,energy,act]=0.0
        return
    def init_Etrace(self):
        self.Etrace=defaultdict(lambda:np.zeros(1))
        for state in range(7):
            for act in ACTION:
                for energy in range(1,self.energy_max+1):
                    self.Etrace[state-3,energy,act]=0
        return

In [2]:
winner=Winner()
Q=winner.sarsalam(500000)
print(Q)

Episode 500000/500000.defaultdict(<function Winner.init_Q.<locals>.<lambda> at 0x7f984c0626a8>, {(-3, 1, <ACTION.high: 'high'>): array([0.]), (-3, 2, <ACTION.high: 'high'>): array([0.]), (-3, 3, <ACTION.high: 'high'>): array([0.]), (-3, 4, <ACTION.high: 'high'>): array([0.]), (-3, 5, <ACTION.high: 'high'>): array([0.]), (-3, 6, <ACTION.high: 'high'>): array([0.]), (-3, 7, <ACTION.high: 'high'>): array([0.]), (-3, 8, <ACTION.high: 'high'>): array([0.]), (-3, 9, <ACTION.high: 'high'>): array([0.]), (-3, 10, <ACTION.high: 'high'>): array([0.]), (-3, 1, <ACTION.low: 'low'>): array([0.]), (-3, 2, <ACTION.low: 'low'>): array([0.]), (-3, 3, <ACTION.low: 'low'>): array([0.]), (-3, 4, <ACTION.low: 'low'>): array([0.]), (-3, 5, <ACTION.low: 'low'>): array([0.]), (-3, 6, <ACTION.low: 'low'>): array([0.]), (-3, 7, <ACTION.low: 'low'>): array([0.]), (-3, 8, <ACTION.low: 'low'>): array([0.]), (-3, 9, <ACTION.low: 'low'>): array([0.]), (-3, 10, <ACTION.low: 'low'>): array([0.]), (-2, 1, <ACTION.high:

In [3]:
B_MAX=10
Q_tb = defaultdict(lambda:np.zeros(2))
for state in range (7):
    for b in range(B_MAX+1):
        Q_tb[state-3,b][0]=Q[state-3,b,ACTION.low]
        Q_tb[state-3,b][1]=Q[state-3,b,ACTION.high]

print(Q_tb)

defaultdict(<function <lambda> at 0x7f984c0627b8>, {(-3, 0): array([0., 0.]), (-3, 1): array([0., 0.]), (-3, 2): array([0., 0.]), (-3, 3): array([0., 0.]), (-3, 4): array([0., 0.]), (-3, 5): array([0., 0.]), (-3, 6): array([0., 0.]), (-3, 7): array([0., 0.]), (-3, 8): array([0., 0.]), (-3, 9): array([0., 0.]), (-3, 10): array([0., 0.]), (-2, 0): array([0., 0.]), (-2, 1): array([10.49485838,  0.03449211]), (-2, 2): array([ 2.61027915, 21.15864094]), (-2, 3): array([48.56684245, 18.38357233]), (-2, 4): array([38.6387672, 56.3108141]), (-2, 5): array([51.03758548, 54.36267831]), (-2, 6): array([59.49309348, 72.52442593]), (-2, 7): array([64.09072221, 50.48467628]), (-2, 8): array([68.75696851, 87.20210184]), (-2, 9): array([59.98736996, 77.33509522]), (-2, 10): array([66.44854606, 92.70244694]), (-1, 0): array([0., 0.]), (-1, 1): array([37.64719722,  1.47396026]), (-1, 2): array([56.77053906,  5.22056683]), (-1, 3): array([119.0220042 ,  73.95711519]), (-1, 4): array([120.28322458,  93.26